In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# pip install mediapipe

In [3]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from tqdm import tqdm
import random
import mediapipe as mp
import cv2
import math

In [54]:
#file_name = "/content/drive/MyDrive/Capp/Dataset/train.pkl"
# file_name = "/content/drive/MyDrive/Capstone/train.pkl"
file_name = "/mnt/d/Research works/sign language/train.pkl"

with open(file_name,'rb') as file:
  input = pickle.load(file)

print(len(input))

#file_name = "/content/drive/MyDrive/Capp/Dataset/index.pkl"
# file_name = "/content/drive/MyDrive/Capstone/index.pkl"
file_name = "/mnt/d/Research works/sign language/index.pkl"

with open(file_name,'rb') as file:
  input_index = pickle.load(file)

print(len(input_index))

1000
1000


In [55]:
input[795]

array([[0.46528506, 0.47905231, 0.48634887, ..., 0.40866986, 0.51176405,
        0.41611767],
       [0.46552891, 0.47927538, 0.48687059, ..., 0.40873903, 0.51004583,
        0.41538209],
       [0.46561375, 0.47930354, 0.4869011 , ..., 0.40594602, 0.50666457,
        0.41111952],
       ...,
       [0.30180004, 0.26837447, 0.26973176, ..., 1.76879084, 1.83976173,
        1.84379184],
       [0.30152467, 0.26845393, 0.26992279, ..., 1.77719951, 1.84889495,
        1.84230125],
       [0.30094016, 0.26803944, 0.26953724, ..., 1.79007387, 1.86223912,
        1.8529706 ]])

In [56]:
input_index[795]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795,
       795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 795, 79

In [57]:
for i in range(1000):
    try:
        print(i, input_index[i].max())
    except:
        break

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 

In [58]:
for i in range(1000):
#  print(input_index[i].shape)
  for j in range(input_index[i].shape[0]):
    if j < 24:
      continue
    elif j < input_index[i].shape[0] - 24:
      input_index[i][j] = input_index[i][j] + 1
    else:
      input_index[i][j] = 0

In [59]:
input_index[900]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901, 901,
       901, 901, 901, 901, 901, 901, 901, 901, 901, 901,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0])

In [60]:
def prepare_data(demo_input,demo_index):
  X = []
  Y = []
  #print(len(demo_input))
  #print(len(demo_index))
  i = 0
  while True:
    seq_len=15
    # if (seq_len+i) >= math.ceil(len(demo_input)*0.7):
    if (seq_len+i) < len(demo_input):
      x,y = [],[]
      x.append(demo_input[i:seq_len+i])
      Y.append(demo_index[seq_len+i])
      X.append(x)
      i+=1
    else:
      break
    #print(Y)
  return np.vstack(X),np.hstack(Y)

In [61]:
k = 0
for i,j in zip(input[:796],input_index[:796]):
    # if (i.shape[0] <350) & (i.shape[0] > 0):
        print(k,input_index[k].max())
        k += 1

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 26
26 27
27 28
28 29
29 30
30 31
31 32
32 33
33 34
34 35
35 36
36 37
37 38
38 39
39 40
40 41
41 42
42 43
43 44
44 45
45 46
46 47
47 48
48 49
49 50
50 51
51 52
52 53
53 54
54 55
55 56
56 57
57 58
58 59
59 60
60 61
61 62
62 63
63 64
64 65
65 66
66 67
67 68
68 69
69 70
70 71
71 72
72 73
73 74
74 75
75 76
76 77
77 78
78 79
79 80
80 81
81 82
82 83
83 84
84 85
85 86
86 87
87 88
88 89
89 90
90 91
91 92
92 93
93 94
94 95
95 96
96 97
97 98
98 99
99 100
100 101
101 102
102 103
103 104
104 105
105 106
106 107
107 108
108 109
109 110
110 111
111 112
112 113
113 114
114 115
115 116
116 117
117 118
118 119
119 120
120 121
121 122
122 123
123 124
124 125
125 126
126 127
127 128
128 129
129 130
130 131
131 132
132 133
133 134
134 135
135 136
136 137
137 138
138 139
139 140
140 141
141 142
142 143
143 144
144 145
145 146
146 147
147 148
148 149
149 150
150 151
151 152
15

In [62]:
#input_lstm_x = []
#input_lstm_y = []
input_lstm = []

for i,j in zip(input[:796],input_index[:796]):
  print(i.shape)
  print(j.shape)
  if (i.shape[0] <350) & (i.shape[0] > 0):
    input_x,input_y = prepare_data(i,j)
    input_tup = (input_x,input_y)
    input_lstm.append(input_tup)
  else:
    continue
  #input_lstm_x.append(input_x)
  #input_lstm_y.append(input_y)

(107, 66)
(107,)
(102, 66)
(102,)
(87, 66)
(87,)
(118, 66)
(118,)
(112, 66)
(112,)
(95, 66)
(95,)
(133, 66)
(133,)
(120, 66)
(120,)
(138, 66)
(138,)
(117, 66)
(117,)
(117, 66)
(117,)
(107, 66)
(107,)
(117, 66)
(117,)
(120, 66)
(120,)
(126, 66)
(126,)
(109, 66)
(109,)
(108, 66)
(108,)
(126, 66)
(126,)
(120, 66)
(120,)
(113, 66)
(113,)
(125, 66)
(125,)
(89, 66)
(89,)
(103, 66)
(103,)
(146, 66)
(146,)
(139, 66)
(139,)
(81, 66)
(81,)
(110, 66)
(110,)
(102, 66)
(102,)
(180, 66)
(180,)
(141, 66)
(141,)
(184, 66)
(184,)
(127, 66)
(127,)
(173, 66)
(173,)
(162, 66)
(162,)
(184, 66)
(184,)
(161, 66)
(161,)
(81, 66)
(81,)
(111, 66)
(111,)
(62, 66)
(62,)
(133, 66)
(133,)
(98, 66)
(98,)
(114, 66)
(114,)
(68, 66)
(68,)
(103, 66)
(103,)
(107, 66)
(107,)
(98, 66)
(98,)
(86, 66)
(86,)
(111, 66)
(111,)
(95, 66)
(95,)
(107, 66)
(107,)
(244, 66)
(244,)
(77, 66)
(77,)
(77, 66)
(77,)
(90, 66)
(90,)
(161, 66)
(161,)
(110, 66)
(110,)
(66, 66)
(66,)
(98, 66)
(98,)
(108, 66)
(108,)
(109, 66)
(109,)
(88, 66)
(88

In [63]:
input_index[0].shape, input[0].shape

((107,), (107, 66))

In [64]:
input_lstm[0][0].shape

(92, 15, 66)

In [65]:
len(input_lstm)

569

In [66]:
type(input_lstm)

list

In [67]:
# device = torch.device('cpu')

In [68]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        self.num_layers = 2
        self.lstm_size = 300

        self.lstm = nn.LSTM(input_size=66, hidden_size=self.lstm_size,
                            num_layers=self.num_layers, batch_first=True,dropout=0.2)
        self.fc1 = nn.Linear(in_features=self.lstm_size, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=797)
        # self.bn = nn.BatchNorm1d(num_features=self.lstm_size)

    def init_state(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.lstm_size).to(device),
                torch.zeros(self.num_layers, batch_size, self.lstm_size).to(device))

    def forward(self, x):
        x, s = self.lstm(x, self.init_state(x.size()[0]))
        x = x[:,-1,:]
        # x = self.bn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x, s

model = LSTM()

In [69]:
# model = model.to(device)
opt = Adam(model.parameters(), lr=0.001)

In [70]:
def train_lstm(data_loader, model):

    model.train()
    training_loss = 0.0 #np.array([])

    for idx, (data, target) in enumerate(data_loader):
        data = data.float().to(device)
        target = target.reshape(len(data),).float().to(device)

        opt.zero_grad()

        pred, (hidden_state, cell_state) = model(data)
        loss = F.cross_entropy(pred, target.long())
        #loss = F.binary_cross_entropy(pred, target)
        loss.backward()

        training_loss += loss.item() # np.append(training_loss, [loss.item()])

        opt.step()

    return training_loss, pred # training_loss.mean(),pred


In [71]:
def val_lstm(data_loader,model):
  val_loss = 0.0
  model.eval()
  #validation_loss = np.array([])

  #for idx,(data,target) in enumerate(data_loader):
  #  print("Data type of data:",type(data))
  #  data = data.float().to(device)
  #  target = target.reshape(len(data),).float().to(device)
  #  output = model(data)
  #  print(type(output))
  #  output = torch.tensor(output)
  #  print(type(output))
  #  loss = F.cross_entropy(output,target.long())
  #  validation_loss = np.append(validation_loss,[loss.item()])
#
  #return validation_loss


  with torch.no_grad():
    for batch, target in tqdm(data_loader):
      batch = batch.float().to(device)
      target = target.to(device)
#
  #  #opt.zero_grad()
#
      output,(_,_) = model(batch)
      print(output)
      #output = torch.tensor(list(output))
      output = torch.stack(list(tuple_of_tensors), dim=0)
      loss = F.cross_entropy(output, target)
  #  #loss.backward()
  #  #opt.step()
#
      val_loss += loss.item()
  #  #if (epoch+1) % 5 == 0:
  #    #print(f"Training loss after {epoch+1} epoch is: {training_loss}")
#
  return val_loss

In [ ]:
for i in range(len(input_lstm)):
    print(i, input_lstm[i][1].max())

In [73]:
device = torch.device('cuda')

In [81]:
max_epoch = 150
epoch_losses = np.array([])
model = model.to(device)
for epoch in tqdm(range(max_epoch)):
  epoch_loss_train = 0.0
  #print(type(input_lstm))
  random.shuffle(input_lstm)
  for i in range(len(input_lstm)):
    train_loader = DataLoader(TensorDataset(torch.tensor(input_lstm[i][0]), torch.tensor(input_lstm[i][1])),batch_size=32,shuffle=False)
    epoch_loss,pred = train_lstm(train_loader,model=model)
    # epoch_losses = np.append(epoch_losses,[epoch_loss])
    epoch_loss_train += epoch_loss
    pred = pred.detach().cpu().numpy()

  if (epoch+1) % 1 == 0:
    print("Epoch: {} and loss: {}".format(epoch+1, epoch_loss_train))

  0%|          | 0/150 [00:00<?, ?it/s]

  1%|          | 1/150 [00:38<1:36:27, 38.84s/it]

Epoch: 1 and loss: 2333.0058513618233


  1%|▏         | 2/150 [01:15<1:32:11, 37.37s/it]

Epoch: 2 and loss: 2336.5886429472403


...

Epoch: 48 and loss: 2001.7046227906546


 33%|███▎      | 49/150 [34:07<1:12:49, 43.26s/it]

Epoch: 49 and loss: 1979.7935392799802


 33%|███▎      | 49/150 [34:08<1:10:22, 41.81s/it]


KeyboardInterrupt: 

In [ ]:
7.202643028972459

In [82]:
#torch.save(model.state_dict(), '/content/drive/MyDrive/Capp/Dataset/hand_sign_model.pth')
torch.save(model.state_dict(), '/mnt/d/Research works/sign language/hand_sign_model.pth')

In [ ]:
test_free = '/content/drive/MyDrive/Capp/TestData/Lal_free.mp4'
test_fall = '/content/drive/MyDrive/Capp/TestData/Lal_fall.mp4'

In [ ]:
def video_data(video):
  mp_drawing = mp.solutions.drawing_utils
  mp_drawing_styles = mp.solutions.drawing_styles
  mp_pose = mp.solutions.pose

  cap = cv2.VideoCapture(video)
  number_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))

  x = []
  y = []
  length = []

  with mp_pose.Pose(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.5) as pose:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          break

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        if not results.pose_landmarks:
          continue
        for idx,landmark in enumerate(results.pose_landmarks.landmark):
          x.append(landmark.x)
          y.append(landmark.y)
  cap.release()
  cv2.destroyAllWindows()
  length = x
  length.extend(y)
  length_arr = np.array(length).reshape(number_of_frames,66)
  return length_arr,number_of_frames

In [ ]:
free,number_of_frames_free = video_data(test_free)
fall,number_of_frames_fall = video_data(test_fall)

In [ ]:
file_name = "/content/drive/MyDrive/Capstone/index.pkl"
with open(file_name,'rb') as file:
  input_index = pickle.load(file)

file_name="/content/drive/MyDrive/Capstone/target_new.pkl"
with open(file_name,'rb') as file:
  target = pickle.load(file)

In [ ]:
idx_free = target.index('free')
idx_fall = target.index('fall')

In [ ]:
index_free= []
index_fall=[]
for i in range(number_of_frames_free):
  if i < 24:
    index_free.append(0)
  else:
    index_free.append(idx_free+1)

for i in range(number_of_frames_fall):
  if i < 24:
    index_fall.append(0)
  else:
    index_fall.append(idx_fall+1)

print(index_free)
print(index_fall)

[]
[]


In [ ]:
def prepare_data(demo_input,demo_index):
  X = []
  Y = []
  #print(len(demo_input))
  #print(len(demo_index))
  i = 0
  while True:
    seq_len=20
    if (seq_len+i) >= len(demo_input):
      break
    x,y = [],[]
    x.append(demo_input[i:seq_len+i])
    Y.append(demo_index[seq_len+i])
    X.append(x)
    i+=1
    #print(Y)
  return np.vstack(X),np.hstack(Y)

In [ ]:
input_free,ind_free = prepare_data(free,index_free)
input_fall,ind_fall = prepare_data(fall,index_fall)

In [ ]:
model = model.to(device)

In [ ]:
model.eval()

LSTM(
  (lstm): LSTM(66, 300, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=300, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=797, bias=True)
)

In [ ]:
val_loader_free = DataLoader(TensorDataset(torch.tensor(input_free), torch.tensor(ind_free)),batch_size=32,shuffle=False)
model.eval()
with torch.no_grad():
  for data,target in tqdm(val_loader_free):
    data = data.float().to(device)
    out_free,(_,_) = model(data)

#out_free = out_free.cpu().numpy()

In [ ]:
xx, (_,_)=model(torch.tensor(input_free).float().to(device))

In [ ]:
xx,(_,_) = model(torch.tensor(input_lstm[1][0]).float().to(device))

In [ ]:
xx[50].argmax()

tensor(0, device='cuda:0')

In [ ]:
input_lstm[1][1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 25, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25])

In [ ]:
xx_fall, (_,_)=model(torch.tensor(input_fall).float().to(device))
xx_fall

In [ ]:
xx_fall[106].argmax()